# Episode state evolution

Notebook that evalutes the solarMED model for some date, and plots the evolution of the states during the episode.

In [1]:
from pathlib import Path
from loguru import logger
import hjson
from solarmed_modeling.utils import data_preprocessing, data_conditioning
from solarmed_modeling.solar_med import (ModelParameters, 
                                         FixedModelParameters, 
                                         FsmParameters,
                                         EnvironmentParameters)
from solarmed_modeling.solar_med.utils import evaluate_model
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot

%load_ext autoreload
%autoreload 2

logger.disable("phd_visualizations.utils")
logger.enable("solarmed_modeling.solar_med.utils")
logger.enable("solarmed_modeling.utils")

data_path: Path = Path("../../data")
src_diagram_path = Path('../../auxiliar_material/solarMED_optimization-Operating modes.svg')

date_str: str = "20230703" # '20230630'
filename_process_data = f'{date_str}_solarMED.csv'
filename_process_data2 = f'{date_str}_MED.csv'

sample_rate = '400s'
sample_rate_numeric = int(sample_rate[:-1])

# Resample figures using plotly_resampler
resample_figures: bool = False


/workspaces/solarMED-modeling/.venv/lib/python3.11/site-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_params" in SolarMED has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


### Pre-processing

In [9]:
output_path: Path = Path(date_str)
output_path.mkdir(exist_ok=True)


In [2]:
cost_params = EnvironmentParameters()
data_paths = [data_path / f"datasets/{filename_process_data}", data_path / f"datasets/{filename_process_data2}"]

with open( data_path / "variables_config.hjson") as f:
    vars_config = hjson.load(f)
    
with open(data_path/"plot_config_validation.hjson") as f:
    plot_config = hjson.load(f)
    
# Load data and preprocess data
df = data_preprocessing(data_paths, vars_config, sample_rate_key=sample_rate)

# Condition data
df = data_conditioning(df, sample_rate_numeric=sample_rate_numeric, vars_config=vars_config, cost_w=cost_params.cost_w, cost_e=cost_params.cost_e)


2024-11-03 12:54:19.104 | INFO     | solarmed_modeling.utils:data_preprocessing:110 - Reading data from 20230703_solarMED.csv


2024-11-03 12:54:19.287 | DEBUG    | solarmed_modeling.utils:process_dataframe:66 - Index([], dtype='object')
2024-11-03 12:54:19.290 | INFO     | solarmed_modeling.utils:process_dataframe:73 - Number of duplicate index values in df: 0
2024-11-03 12:54:19.315 | INFO     | solarmed_modeling.utils:data_preprocessing:130 - Reading data from 20230703_MED.csv
2024-11-03 12:54:19.638 | DEBUG    | solarmed_modeling.utils:process_dataframe:66 - Index([], dtype='object')
2024-11-03 12:54:19.639 | INFO     | solarmed_modeling.utils:process_dataframe:73 - Number of duplicate index values in df: 0
2024-11-03 12:54:19.643 | DEBUG    | solarmed_modeling.utils:data_preprocessing:149 - Common columns in both dataframes: Index(['FT-AQU-100', 'TT-AQU-107a', 'CT-DES-001', 'FT-DES-003', 'FT-DES-005',
       'SW2TC1', 'TE-DES-015', 'TT-DES-030'],
      dtype='object'), dropping them from the auxiliary dataframe.
2024-11-03 12:54:19.715 | WARNING  | solarmed_modeling.utils:data_preprocessing:173 - Removing 

In [3]:
dfs_mod, stats = evaluate_model(
    df=df, sample_rate = sample_rate_numeric,
    model_params = ModelParameters(), 
    fixed_model_params = FixedModelParameters(),
    fsm_params=FsmParameters(),
    env_params=EnvironmentParameters(),
    alternatives_to_eval = ["constant-water-props"],
)

# fig = experimental_results_plot(plot_config, df, df_comp=dfs_mod[0], vars_config=vars_config, resample=resample_figures)

# fig.show(
#     config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
# )


2024-11-03 12:54:24.199 | INFO     | solarmed_modeling.solar_med.utils:evaluate_model:56 - Starting evaluation of alternative constant-water-props. Sample rate = 400 s


Authorization required, but no authorization protocol specified
/workspaces/solarMED-modeling/src/solarmed_modeling/solar_med/__init__.py:1332: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([df, data], ignore_index=True) if df is not None else data
2024-11-03 12:54:31.033 | INFO     | solarmed_modeling.solar_med.utils:evaluate_model:139 - Finished evaluation of alternative constant-water-props. Elapsed time: 6.832 s, MAE: 6.54 ºC


In [4]:
df_mod = dfs_mod[0]


## Episode state evolution visualization

In [5]:
from solarmed_modeling.visualization.fsm.state_evolution import plot_episode_state_evolution
from solarmed_modeling.fsms import SfTsState, MedState

fig = plot_episode_state_evolution(df_mod, subsystems_state_cls=[SfTsState, MedState], show_edges=False)

fig


FigureWidget({
    'data': [{'hoverinfo': 'text',
              'line': {'color': 'rgb(50,50,50)', 'width': 0.5},
              'marker': {'color': '#ff7800', 'size': 20, 'symbol': 'circle-dot'},
              'mode': 'markers',
              'name': 'states',
              'text': array(['IDLE', 'HEATING_UP_SF', 'SF_HEATING_TS', ..., 'HEATING_UP_SF',
                             'SF_HEATING_TS', 'RECIRCULATING_TS'], dtype=object),
              'type': 'scatter',
              'uid': '08d3b647-14cf-471d-a2a6-86d1989b75b0',
              'x': [0, 0, 0, 0, 4, 4, 4, 4, 8, 8, 8, 8, 12, 12, 12, 12, 16, 16,
                    16, 16, 20, 20, 20, 20, 24, 24, 24, 24, 28, 28, 28, 28, 32, 32,
                    32, 32, 36, 36, 36, 36, 40, 40, 40, 40, 44, 44, 44, 44, 48, 48,
                    48, 48, 52, 52, 52, 52, 56, 56, 56, 56, 60, 60, 60, 60, 64, 64,
                    64, 64, 68, 68, 68, 68, 72, 72, 72, 72, 76, 76, 76, 76, 80, 80,
                    80, 80, 84, 84, 84, 84, 88, 88, 88

### Episode state evolution with facility diagram animation

In [10]:
# Hihglight state evolution to generate a GIF
from cairosvg import svg2png
from copy import deepcopy
from lxml import etree
from phd_visualizations.utils import stack_images_vertically
from solarmed_modeling.visualization.fsm.state_evolution import plot_episode_state_evolution
from solarmed_modeling.visualization.fsm.facility_diagram import SolarMedStateVisualizer
from solarmed_modeling.fsms import SfTsState, MedState

logger.disable("phd_visualizations.diagrams")

plot_path = output_path / "plots"
plot_path.mkdir(exist_ok=True)

diagram_path = output_path / "diagrams"
diagram_path.mkdir(exist_ok=True)

height=600
width=1800

with open(src_diagram_path, 'r') as f:
    diagram_file = etree.parse(f)

fig = plot_episode_state_evolution(df_mod, subsystems_state_cls=[SfTsState, MedState], show_edges=False)

for step_idx in fig.layout.xaxis.tickvals:
    # Generate plot
    plot_episode_state_evolution(df_mod, subsystems_state_cls=[SfTsState, MedState], 
                                 show_edges=False, highligth_step=step_idx,
                                 width=width, height=height).write_image(plot_path / f"step_{step_idx:03}.png")
    
    # Generate diagram
    sf_state, ts_state, med_state = df_mod.loc[df_mod.index[step_idx], ['sf_state', 'ts_state', 'med_state']]
    solar_med_state = SolarMedStateVisualizer(sf_state=sf_state.value, ts_state=ts_state.value, med_state=med_state.value)

    diagram = solar_med_state.create_state_diagram(deepcopy(diagram_file))
    diagram.write( diagram_path / f'step_{step_idx:03}.svg')
    
    svg2png(url=str(diagram_path / f'step_{step_idx:03}.svg'), write_to=str(diagram_path / f'step_{step_idx:03}.png'), 
            output_width=width, output_height=height, dpi=300, background_color='white')

    stack_images_vertically(plot_path / f"step_{step_idx:03}.png", diagram_path / f"step_{step_idx:03}.png", output_path / f"step_{step_idx:03}.png")


Generating the GIF requires having installed `imagemagick`

In [ ]:
import subprocess
import os

# Newer systems
# magick *.png -morph 3 -delay 50 -loop 0 output.gif
# Older systems
# convert *.png -morph 3 -delay 50 -loop 0 output.gif

# ffmpeg -framerate 3 -pattern_type glob -i '*.png' output.gif

# Change directory to output_path
os.chdir(output_path)
subprocess.run(["magick", "*.png", "-morph", "3", "-delay", "50", "-loop", "0", "output.gif"], shell=True)
